In [36]:
import pandas as pd
import sqlalchemy import create_engine
import psycopg2

In [31]:
%reload_ext sql
%config SqlMagic.displaylimit = 10

In [39]:
%sql sqlite://

UsageError: Line magic function `%sql` not found.


## Introduction

### Outline
1. Intro to Jupyter notebooks
2. Into to SQL Magic
3. Overview of the exercise

LET'S LEARN SQL!

### Table Schema
The below tables form the basis of a DVD store's operation. Maybe we'll learn what happened to Blockbuster through this exercise? 

<img src="files/dvd-rental-sample-database-diagram.png">

### The above diagram illustrates the relationship between tables in the database. Listed below is information for each table:

**actor** – actors data including first name and last name.  
**film** – film data such as title, release year, length, rating  
**film_actor** – the relationships between films and actors.  
**category** – film’s categories data.  
**film_category** - the relationships between films and categories.  
**store** – contains the store data including manager staff and address.  
**inventory** – inventory data.  
**rental** – rental data.  
**payment** – customer’s payments.  
**staff** – staff data.  
**customer** – customers data.  
**address** – address data for staff and customers  
**city** –  city names.  
**country** –  country names.  

### In this exercise, we will work with five tables:
1. rental
2. customer
3. payment
4. inventory
5. film

## Section 1: Data Validation  (High Level)

### Outline
**Technical**   
- Primary Keys
    - Duped Primary Keys

**Story**   
- Familiarize with customer, payments, rental + respective join keys

**Content**  
When working with a new data source, it is crucial to QA before integrating the source into your development or analysis. This section will provide a framework for performing high-level checks with common queries.

### Identify Primary Keys
In each table, there should be a field (or combination of fields) that uniquely identifies a row. These unique identifiers are referred to as the table's **primary key**. In the schema above, the primary key for each table is indicated by a * next to the field name (notice that film_category and film_actor each have a primary key with two fields). However, documentation can be incorrect, so it's necessary to verify before moving forward.    

But how do we do this? We can compare a distinct value count for a given field (or combination of fields) to the total number of rows in the table. Let's do this with the rental table. 

### Example Query 1:

In [5]:
%%sql
-- Investigating distinct values for each field to determine what identifies a unique row 
SELECT
    COUNT(*) as number_of_rows,
    COUNT(DISTINCT rental_id) as rental_id_count,
    COUNT(DISTINCT rental_date) as rental_date_count,
    COUNT(DISTINCT inventory_id) as inventory_id_count, 
    COUNT(DISTINCT customer_id) as customer_id_count,
    COUNT(DISTINCT staff_id) as staff_id_count, 
    COUNT(DISTINCT last_update) as last_update_count
FROM rental

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
1 rows affected.


number_of_rows,rental_id_count,rental_date_count,inventory_id_count,customer_id_count,staff_id_count,last_update_count
16044,16044,15815,4580,599,2,3


After running the above query, we are able to confirm that **rental_id is the primary key for the rental table**, as the total number of rows matches the number of distinct rental_ids. However, this query also provides other interesting information:

1. We have 599 customers, but 16,044 transactions! People sure liked DVD rentals in 2006. I wonder what sort of disruptive technologies appeared in the late 2000s to change this...

2. There are only 2 staff members who have autorized rentals (work-life balance doesn't exist in this database).

3. There are 3 different instances of last_update_count, indicating that rows in the table have been modified at different points.

### Test Query 1.1: In the cell below, write a query to determine the primary key for the payment table. 

In [6]:
%%sql
select

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
1 rows affected.


[()]

### Question 1.1: What is the primary key for the payment table? Assign a string to the answer variable and run the cell.

In [7]:
answer = "payment_id"
PaymentPrimaryKeyQuestion(answer)

Correct!


### Test Query 1.2: In the cell below, write a query to determine the primary key for the customer table.

In [8]:
%%sql
select

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
1 rows affected.


[()]

### Question 1.2: What is the primary key for the customer table? Assign a string to answer variable and run the cell.

In [9]:
answer = "customer_id"
CustomerPrimaryKeyQuestion(answer)

Correct!


## Section 2: Data Validation (Data Types)

### Outline
**Technical**    
- String Data Type
- Date Data Type

**Story**
- Familiarize with film + inventory (from strings)
- Highlight date considerations for customer, payments, and rental

### Content  
Now we know the primary keys for the rental, payment, and customer tables. But what are these fields? Let's investigate the rental table's data types. The syntax for describing data types is dependent on the type of SQL you are using. In this tutorial, we are using **PostgreSQL**.   

**NOTE:** If you're working in Databricks, you can use the SQL command "describe [table_name]" to output the schema.

#### TO ADD: List of SQL variations that use the query below

### Example Query 2:

In [10]:
%%sql
select column_name, data_type 
from information_schema.columns
-- Change the table_name to the table you want to investigate
where table_name = 'rental';

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
7 rows affected.


column_name,data_type
rental_id,integer
rental_date,timestamp without time zone
inventory_id,integer
customer_id,smallint
return_date,timestamp without time zone
staff_id,smallint
last_update,timestamp without time zone


In [11]:
# TO DO: Modify the the payment table/customer table so the join keys are different across different tables.
# i.e. rental_table is an integer in one table and a string in another table
# other note: could also add leading 0s to some of the join keys

### Test Query 2: In the cell below, write a query to determine the data types for the payment table.

In [23]:
%%sql
select

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
1 rows affected.


[()]

### Question 2: What is the data type for the amount field in the payment table? Assign a string to the answer variable and run the cell.

In [24]:
answer = "numeric"
PaymentDataTypeQuestion(answer)

Correct!


### Strings
A string is a sequence of characters, either as a  constant or as a variable. Strings are used to represent text, but numbers and dates may still be string types.    
     
#### Example Strings: 
“String”, “Str!nG2!”, “34578”, “U84-32-12-44”, “2018-12-10”  
  
### Substrings
When working with strings, you may want to investigate certain portions of strings in your table (AKA substrings). For example, **“Cascade”** is a substring of **“Cascade Data Labs”**.  
  
**NOTE:**  
There are many functions in SQL used to pull substrings. The most common are **LEFT(string, length)** (pulls from the string’s left side) , **RIGHT(string, length)** (pulls from the string’s right side), and **SUBSTRING**(**expression** (string to match for), **start** (first character is 1), **length** (inclusive of start)).    

### String Matching
When working with data, you often want to filter your output to certain instances of strings. For example, you may want to filter out specific products or filter for a certain customer. We will investigate the different methods in string matching through the films and inventory tables.

**WHERE-RELATIONAL OPERATOR Clause**  
The WHERE-RELATIONAL OPERATOR Clause is the simplest string matching method (and is not limited to the string data type), but also the least robust. 

**TIP:** For exact string matching of alphabetical strings, it is often easiest to user **UPPER(** string **)** (function to make entire string uppercase) on the word you are searching for, as string data can be inconsistent.

**Film Table**
To get a sense for the information in the film table, let's pull all the fields for the film "Airport Pollock". 

### TO DO: Figure out a way to articulate why single cases are useful in QA, specifically in tracking issues

### Example Query 3:

In [25]:
%%sql
SELECT * 
FROM film 
-- WHERE-RELATIONAL OPERATOR CLAUSE
WHERE UPPER(title) = 'AIRPORT POLLOCK'

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
1 rows affected.


film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features
8,Airport Pollock,A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India,2006,1,6,4.99,54,15.99,R,2013-05-26 14:50:58.951000,['Trailers']


The above output reveals some interesting things about this film table:

    1. This DVD store contains films where moose are protagonists and monkeys are anti-heroes.
    2. Values for the special features field are arrays.

### Test Query 3:  In the cell below, write a query to pull the description, release_year, and rental_rate for the film "african egg". Begin the query below "answer <<" 

In [26]:
%%sql 
answer << 
select description, release_year, rental_rate from film where UPPER(title) = 'AFRICAN EGG'

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
1 rows affected.
Returning data to local variable answer


### Question 3: Run the below cell after you've run Test Query 3

In [20]:
# Check your query by running this cell
''' 
NOTE: The output of a dataframe contains an index that uniquely identifies rows,
which is different than the output of a SQL query. 

This does not mean that it serves as the table's primary key.
'''
AfricanEggQuestion(answer)

Correct!


**WHERE-LIKE Clause**  
In that last query, we pulled rows with the title "african egg". But what if we we wanted to pull rows where the title contains the word "africa" (or an extension of the word like african). In order to accomplish this, you need the LIKE clause, which allows for pattern matching. The LIKE clause uses wildcards to identify patterns. There are two wildcards used in conjunction with the LIKE operator:   
    1. % The percent sign represents zero, one, or multiple characters       
    2. _ The underscore represents a single character
  
**TIP**: When pattern matching, make your search pattern as specific as possible. The more general your pattern is, the more likely it will return unexpected outputs.

### Example Query 4:

In [22]:
%%sql
SELECT film_id, title FROM film WHERE UPPER(title) LIKE '%AFRICA%'

 * postgresql+psycopg2://andrewmahler@localhost:5432/sqltool
3 rows affected.


film_id,title
5,African Egg
472,Italian African
637,Open African


## Section 3: Joins + Subqueries
Technical Points:
- Basic Joins
- Dupes that result from joins
- Left Join vs Inner Join vs Anti Join
- Subqueries
    - Optimizing joins/only join what's needed via subqueries
    
Story Points:
- Connect film + inventory
- Connect rental, payment, + customer
- General outline for our analysis

## Section 4: Conditional Logic
Technical Points:
- Complicated Logic
- Case Statements
    - Flags
- DISTINCT Case

Story Points:
- Outline conditions that will feed into final analysis framework

In [ ]:
# Does the above section include case statements?

## Section 5: Aggregation, Grouping, and Windowing Functions

Technical Points:
- Limitations of granularity: rolling up is NOT reversible
- Aggregate functions
- Aggregate functions with case statements
- Partition functions

Story Points:
- Final aggregation of analysis tables
- Add visuals afterwards
- Ends "Stage 1" of the SQL Tool

## Section 6: Advanced Concepts

## APPENDIX:

### Julie Join Notes: 

#### Joins 
So far, we have explored how to query information from a single table and transform the values within that table. However, in the situation where the information we need does not exist in our current table, a join is needed to append the appropriate data onto the data set. 

#### Joins for Data Creation 
Types of Joins
Different types of joins will yield different result tables. It is recommended that the final result table is planned out and thought through before performing the join. Let us use the tables mentioned above as examples (pos_line_item and cust_attributes). 

Note that to join any two tables together, a join key should be used so that the table knows which rows to append together. The join key is typically a field in one table that is supposed to match to a field in the second table. Let’s assume we have two tables, one on the left and one on the right. 

#### Examples once data is added 
An **inner** join returns records that have matching values in both tables

A **left** join returns all records from the left table and corresponding matched values from the right table 

A **right** join returns all records from the right table and corresponding matched values from the left table
Note that this is essentially the same as a left join but with the right table

An **outer** join returns all records from both tables, but will match the values if the join key is validated

An **anti** join returns the records if there is no match in either table   

#### Join Keys
##### Equality vs. inequality  
Join keys are the most essential part of a join. A typical join key, as noted in the previous section, consists of a statement that connects one field in the left table to one field in the right table. Typical join statements use equality, but using inequality in join statements is possible. 

**Joins for Data Filtering**   
Joins are more useful than just appending on new data from different sources. They are also effective in optimizing a slow query when rows need to be filtered out (as opposed to using a where clause in conjunction with the “in” function). There are two joins that can help you accomplish this: (1) inner joins and (2) anti joins. 

Examples once data is added

**Inner Join**

**Anti Join**   
Tips for Joins   
Extra caution is needed when joining on a string join key. It is possible that these strings are capitalized differently and encoded differently. It is best practice to check the join key in both tables before performing the join and if needed, mutate the join key so that they match.

Examples once data is added
For example, for trim(upper(name)) = trim(upper(campaign))

Before performing a join, think about what you want the final data set to look like. Write out all the features and which tables you need to retrieve those fields from before you join tables together. This will help you be organized and write cleaner code. 

Try to alias tables with a small identifier that is descriptive of the table you are querying from. Tag each field you select with the table identifier so that each field is pulled exactly from the table you want.

